In [1]:
import pandas as pd
import csv
import numpy as np
import datetime 
import pdb
import time

FILENAME_D = 'data/consumption_d.csv'
FILENAME_N = 'data/consumption_n.csv'
FILENAME_OUT = 'data/full_data.csv'
FILENAME_TOTAL_OUT = 'data/total_cons.csv'

In [105]:
#final handle_nans
FILENAME_OUT = 'data/full_data2'

def load_tariff():
    return pd.read_csv('data/tariff_d.csv',index_col=0)['Price']

def interpolate_nans(df,maxNans): #replace nans with interpolation
    df = df_without_n_nans(df,maxNans)
    df = df.replace(0,np.nan)
    df = df.interpolate()
    return df

def remove_consecutive_zeros(df,n): # link for original code: https://stackoverflow.com/a/24892274
    final_list=[]
    for customer in df.columns[1:]:
        as_np=df[customer].values
        iszero = np.concatenate(([0], np.equal(as_np, 0).view(np.int8), [0]))
        absdiff = np.abs(np.diff(iszero))
        ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
        range_list=[]
        for item in ranges:
            range_list.append(item[1]-item[0])
        try:
            final_list.append(max(range_list))
        except ValueError:
            final_list.append(0)
    idx_remove = np.where(np.array(final_list)>7)[0]+1 
    return df.drop(df.columns[idx_remove], axis=1).reset_index(drop = True)  
            
def add_tof(df,tof):
    df.loc[-1] = df.shape[1]*[tof]  # adding a row
    df.index = df.index + 1  # shifting index
    df = df.sort_index()  # sorting by index
    return df

def save_data(filenameOut,maxNans = 7):
    dfN = pd.read_csv('data/consumption_n.csv').fillna(0)
    dfD = pd.read_csv('data/consumption_d.csv').fillna(0)
    #drop last timestamp to have whole days
    dfN = dfN.iloc[:-1,:]
    dfD = dfD.iloc[:-1,:]
    #ditch timestamps before 01.01
    dfN['GMT'] = pd.to_datetime(dfN['GMT'])
    dfD['GMT'] = pd.to_datetime(dfD['GMT'])
    startDate=pd.to_datetime('01/01/13 00:00')
#     endDate=pd.to_datetime('31/12/13 23:30')
    dfN = dfN[dfN['GMT'] >= startDate].reset_index(drop = True)
    dfD = dfD[dfD['GMT'] >= startDate].reset_index(drop = True)
    
    dfN = remove_consecutive_zeros(dfN,7)
    dfD = remove_consecutive_zeros(dfD,7)
    dfN = dfN.interpolate()
    dfD = dfD.interpolate()
    dfN = add_tof(dfN,0)
    dfD = add_tof(dfD,1)
    df = pd.concat([dfN,dfD],axis=1)
    df = df.T
    df = df.reset_index(drop=True)
    df.columns = df.iloc[0]
    df = df.reindex(df.index.drop(0)).reset_index(drop=True)
    df = df.rename(columns = {0:'tof'})
    df.to_csv(filenameOut,index=False)
    return df

def aggregated_consumptions(df):
    last_timestamp = df.columns.get_loc('2013-12-31 23:30:00')
    tariff = load_tariff()
    total_1 = df[df['tof']==1].iloc[:,:last_timestamp+1].sum()[1:]
    total_0 = df[df['tof']==0].iloc[:,:last_timestamp+1].sum()[1:]
    total = pd.concat([total_1,total_0,tariff],axis=1).reset_index(drop=True)
    total.columns = ['tof','no_tof','tariff']
    total.to_csv(FILENAME_TOTAL_OUT,index=False)
    return total

In [ ]:
# Example run

dataset = save_data(FILENAME_OUT)
total_cons = aggregated_consumptions(dataset)